In [ ]:
 !pip install transformers

     |████████████████████████████████| 778kB 4.2MB/s 
     |████████████████████████████████| 890kB 13.1MB/s 
     |████████████████████████████████| 1.1MB 19.5MB/s 
     |████████████████████████████████| 3.0MB 31.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=88ea39c9478eeb9e0c44157a03d567a7fb1b9bdea382c13785d6ef7a69a8604a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
import pandas as pd
from transformers import BertTokenizer
from transformers.tokenization_utils import TensorType
import torch

drive.mount('/content/gdrive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
import os
from transformers import BertForSequenceClassification, BertTokenizer

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained("/content/gdrive/My Drive/Colab Notebooks/10_version/")
tokenizer = BertTokenizer.from_pretrained("/content/gdrive/My Drive/Colab Notebooks/10_version/")

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from transformers.tokenization_utils import TensorType
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
from sklearn.metrics import matthews_corrcoef
import torch


def getScores(sentences, labels):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 64,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          truncation = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Set the batch size.  
  batch_size = 32  

  # Create the DataLoader.
  prediction_data = TensorDataset(input_ids, attention_masks, labels)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

  # Prediction on test set

  print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []

  # Predict 
  for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  # Combine the results across all batches. 
  flat_predictions = np.concatenate(predictions, axis=0)

  # For each sample, pick the label (0 or 1) with the higher score.
  flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

  # Combine the correct labels for each batch into a single list.
  flat_true_labels = np.concatenate(true_labels, axis=0)

  #print(len(flat_true_labels))

  cnt = 0
  for i in range (len(flat_predictions)):
    if flat_predictions[i] != flat_true_labels[i]:
        cnt = cnt + 1
  #print(cnt)

  pos = []
  neg = []
  falsePos = []
  falseNeg = []


  for i in range (len(sentences)):
    if flat_predictions[i] == 1 and flat_true_labels[i] == 1:
      pos = np.append(pos,sentences[i], axis=None)
    elif flat_predictions[i] == 0 and flat_true_labels[i] == 0:
      neg = np.append(neg,sentences[i], axis=None)
    elif flat_predictions[i] == 1 and flat_true_labels[i] == 0:
      falsePos = np.append(falsePos,sentences[i], axis=None)
    elif flat_predictions[i] == 0 and flat_true_labels[i] == 1:
      falseNeg = np.append(falseNeg,sentences[i], axis=None)
  accuracy = 0
  precision = 0
  recall = 0
  f1 = 0
  try:
    accuracy = (len(pos) + len(neg)/len(sentences))
  except:
    print("No Accuracy")
  try:
    precision = len(pos)/(len(pos) + len(falsePos))
  except:
    print("No Precision")
  try:
    recall = len(pos)/(len(pos) + len(falseNeg))
  except:
    print("No Recall")
  try:
    f1 = (2*(recall*precision))/(recall + precision)
  except:
    print("No F1 Score")


  print("Accuracy is: " + str(accuracy))
  print("Precision is: " + str(precision))
  print("Recall is: " + str(recall))
  print("F1 Score is: " + str(f1))

  print(flat_predictions)
  print(flat_true_labels)
  print(sentences)
  print(len(sentences))

  return flat_predictions

def main():
  values = ["Keshav is a super duper cool cat", "We have recently shown that the accumulation of diverse viral and cellular membrane proteins in the ER activates the higher eukaryotic transcription factor NF-kappaB . ++++ We show that ER overload mediated NF-kappaB activation but not TNF stimulated NF-kappaB induction can be inhibited by the intracellular Ca2+ chelator TMB-8 . ++++ We therefore propose that ER overload activates NF-kappaB by causing Ca2+ release from the ER . ++++ In contrast , the ER stress signal which activates the NF-kappaB pathway is not known ."]
  labels = []
  for i in range (len(values)):
    labels = np.append(labels,0, axis=None)
  print(getScores(values,labels))

if __name__ == "__main__":
  main()

Predicting labels for 2 test sentences...
No Recall
No F1 Score
Accuracy is: 0.5
Precision is: 0.0
Recall is: 0
F1 Score is: 0
[0 1]
[0. 0.]
['Keshav is a super duper cool cat', 'We have recently shown that the accumulation of diverse viral and cellular membrane proteins in the ER activates the higher eukaryotic transcription factor NF-kappaB . ++++ We show that ER overload mediated NF-kappaB activation but not TNF stimulated NF-kappaB induction can be inhibited by the intracellular Ca2+ chelator TMB-8 . ++++ We therefore propose that ER overload activates NF-kappaB by causing Ca2+ release from the ER . ++++ In contrast , the ER stress signal which activates the NF-kappaB pathway is not known .']
2
[0 1]
